In [ ]:
import math
import sys
import os

import ipyplot

import numpy as np
import pandas as pd

import insightface
import insightface.app as app

#plt.imshow(f2)
#import urllib
import PIL
#import timm
import torch
import shapely
import cv2

from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

from shapely.geometry import *
from pathlib import Path
from ipywidgets import widgets

from matplotlib import pyplot as plt

In [ ]:

class Face(object):
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/home/taylor/AutogluonModels/avatars_face")
    #predictor.evaluate(test_data)
    app = FaceAnalysis() # enable detection model only
    app.prepare(ctx_id=0, det_size=(224, 224))
    def __init__(self,filepath):
        self.filepath=filepath
        img = cv2.imread(self.filepath)
        img = cv2.resize(img,(224,224))
        self.faces = Face.app.get(img)
        self.blur = cv2.Laplacian(img, cv2.CV_64F).var()
        self.proba = self.predictor.predict_proba(data=self.filepath,realtime=True)
    def is_zero_faces(self):
        if self.faces:
            return len(self.faces)==0
        else:
            return True
    def major_face_angle(self):
        land = self.faces[0]['landmark_3d_68']
        low_mid = land[8]
        up_mid = land[27]
        #land2d = filepath[2][0]['landmark_2d_106']
        #low_mid2d = land2d[16] #num 17 
        #up_mid2d = land2d[51] #num 52
        y = abs(up_mid[1]-low_mid[1])
        x = abs(up_mid[0]-low_mid[0])
        #y2d = abs(up_mid2d[1]-low_mid2d[1])
        #x2d = abs(up_mid2d[0]-low_mid2d[0])
        #angel2d = math.atan2(y,x)
        angel = math.atan2(y,x)
        a2 = angel/math.pi*180
        return a2
    def area_of_face(self,face_index):
            land1 = self.faces[face_index]['landmark_2d_106'][0:33]
            area1 = Polygon(land1).area
            return area1
    def face_ratio_by_multi_faces(self):
        if len(self.faces)>=2:
            return self.area_of_face(1)/self.area_of_face(0)
        else:
            return None
    def good_for_port(self):
        if self.proba[0][0] > 0.6:
            return True,"ByModel"
        elif self.proba[0][0] < 0.4:
            return False,"ByModel"
        elif self.blur < 40:
            return False,"Blur"
        elif len(self.faces) == 0:
            return False,"ZeroFaces"
        elif len(self.faces) >= 2:
            return False,"Two+Faces"
        elif self.major_face_angle() < 50:
            return False,"SlantFace"
        else :
            return True,"GoodFace"

In [ ]:
import pathlib
fulllist = pathlib.Path(".").glob("avatars224/passed/*.*")
z = list(str(a) for a in fulllist)
for f in z:
    face = Face(f)
    a,b = face.good_for_port()
    if not a:
        print(f,b)

In [ ]:
import pathlib
fulllist = pathlib.Path(".").glob("avatars224/rejected/*.*")
z = sorted(list(str(a) for a in fulllist))
for f in z[:100]:
    face = Face(f)
    a,b = face.good_for_port()
    if a:
        print(f,b)

In [ ]:
#create dataset

import csv
import os
os.chdir("avatars224")
with open('train.csv', 'w', newline='') as f:
    lable_to_idx={
        "passed":0,
        "rejectd":1
    }
    writer = csv.writer(f)
    writer.writerow(["image","label"])
    for img in glob.glob("/home/taylor/avatars224/train/passed/*.png"):
        writer.writerow([img,0])
    for img in glob.glob("/home/taylor/avatars224/train/rejected/*.png"):
        writer.writerow([img,1])
    
with open('test.csv', 'w', newline='') as f:
    lable_to_idx={
        "passed":0,
        "rejectd":1
    }
    writer = csv.writer(f)
    writer.writerow(["image","label"])
    for img in glob.glob("/home/taylor/avatars224/test/passed/*.png"):
        writer.writerow([img,0])
    for img in glob.glob("/home/taylor/avatars224/test/rejected/*.png"):
        writer.writerow([img,1])
    
    

In [ ]:
#load dataset

os.chdir("/home/taylor")
train_data = pd.read_csv("avatars224/train.csv")
test_data = pd.read_csv("avatars224/test.csv")

In [ ]:
#train the model

from autogluon.multimodal import MultiModalPredictor
from autogluon.multimodal.utils.misc import shopee_dataset

predictor = MultiModalPredictor(label='label').fit(train_data=train_data)
predictions = predictor.predict(test_data)

In [ ]:
# done the predictor
from autogluon.multimodal import MultiModalPredictor


predictor = MultiModalPredictor(label='label').fit(train_data=train_data)
predictions = predictor.predict(test_data)

In [ ]:
# doing some check
from IPython import display
import pathlib
fulllist = pathlib.Path(".").glob("avatars224/passed/*.*")
z = list(str(a) for a in fulllist)
for f in z[:100]:
    face = Face(f)
    a,b = face.good_for_port()
    if not a:
        print(f,b,predictor.predict(data=f,realtime=True),predictor.predict_proba(data=f,realtime=True))
        ipyplot.plot_images([f,])
        
        

In [ ]:
# doing some check
import os
os.chdir("/home/taylor")
import warnings
warnings.filterwarnings('ignore')

from IPython import display
import pathlib
fulllist = pathlib.Path(".").glob("avatars224/test/rejected/*.*")
z = list(str(a) for a in fulllist)
for f in z[:100]:
    face = Face(f)
    a,b = face.good_for_port()
    if a:
        print(f,b,predictor.predict(data=f,realtime=True),predictor.predict_proba(data=f,realtime=True))
        ipyplot.plot_images([f,])